# Ingesting of Myers' QSO catalog (v1.0)

In [1]:
# imports

## Grabbing the catalog

http://faraday.uwyo.edu/~admyers/scratch/forjoe/GTR-ADM-QSO-master-wvcv.fits.gz
    
    by JXP on 5 August 2016

## Inspect

In [4]:
ADM_file = os.getenv('RAW_IGMSPEC')+'/Myers/GTR-ADM-QSO-master-wvcv.fits.gz'
ADM_qso = Table.read(ADM_file)

In [3]:
ADM_qso[0:5]

RA,DEC,ZBEST [19],SOURCEBIT,SDSS_UFLG,AGES_QSO,AGES_CODE06,KDE_ZPHOTLO,KDE_ZPHOTHI,KDE_ZPHOTPROB,KDE_LOWZORUVX,XDQSOZ_PEAKPROB,XDQSOZ_PEAKFWHM,XDQSOZ_NPEAKS,YAPERMAG3,JAPERMAG3,HAPERMAG3,KSAPERMAG3,YAPERMAG3ERR,JAPERMAG3ERR,HAPERMAG3ERR,KSAPERMAG3ERR,ZPHOTMINJHK,ZPHOTBESTJHK,ZPHOTMAXJHK,ZPHOTPROBJHK
float32,float32,float32,int32,int16,int16,int32,float32,float32,float32,int16,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
0.027228,0.515341,1.82383 .. 0.0,34305,0,0,0,1.62,2.11,0.719,1,0.983058,0.1924,1,-9.99999e+08,19.1973,18.8874,18.3277,-9.99999e+08,0.0942081,0.187276,0.172265,1.45,1.825,1.95,0.998998
0.0339,0.276301,1.83638 .. 0.0,33281,0,0,0,1.76,2.14,0.946,1,0.0,0.0,0,-9.99999e+08,18.3689,17.716,17.1701,-9.99999e+08,0.0459005,0.0655821,0.0615399,1.7,1.875,1.95,0.996776
0.038604,15.2985,1.19748 .. 0.0,139777,1,0,0,1.1,1.53,0.912,1,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.039089,13.9384,2.2342 .. 2.239,403073,1,0,0,0.93,1.44,0.634,1,0.827387,0.1976,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.039271,-10.4644,1.85301 .. 0.0,132609,0,0,0,1.45,2.06,0.739,1,0.939097,0.3484,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
hdu = fits.open(ADM_file)

Filename: /u/xavier/Raw_IGMspec/Myers/GTR-ADM-QSO-master-wvcv.fits.gz
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       4   ()              
1                BinTableHDU     70   1969183R x 26C   [E, E, 19E, J, I, I, J, E, E, E, I, E, E, I, E, E, E, E, E, E, E, E, E, E, E, E]   


In [11]:
head0 = hdu[0].header
head0

SIMPLE  =                    T /Dummy Created by MWRFITS v1.8                   
BITPIX  =                    8 /Dummy primary header created by MWRFITS         
NAXIS   =                    0 /No data is associated with this header          
EXTEND  =                    T /Extensions may (will!) be present               

In [13]:
head1 = hdu[1].header
head1['DATE']

'Sat Oct 10 07:10:38 2015'

## Redshifts

In [ ]:
# Run zbest_myers()